In [0]:
!wget https://www.machinehack.com/wp-content/uploads/2020/04/MessagePolarity_ParticipantsData.zip

--2020-05-03 07:45:05--  https://www.machinehack.com/wp-content/uploads/2020/04/MessagePolarity_ParticipantsData.zip
Resolving www.machinehack.com (www.machinehack.com)... 107.180.71.20
Connecting to www.machinehack.com (www.machinehack.com)|107.180.71.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206479 (202K) [application/zip]
Saving to: ‘MessagePolarity_ParticipantsData.zip’

MessagePolarity_Par 100%[===================>] 201.64K   296KB/s    in 0.7s    

2020-05-03 07:45:06 (296 KB/s) - ‘MessagePolarity_ParticipantsData.zip’ saved [206479/206479]



In [0]:
!unzip MessagePolarity_ParticipantsData.zip

Archive:  MessagePolarity_ParticipantsData.zip
   creating: MessagePolarity_ParticipantsData/
  inflating: MessagePolarity_ParticipantsData/Test.csv  
  inflating: __MACOSX/MessagePolarity_ParticipantsData/._Test.csv  
  inflating: MessagePolarity_ParticipantsData/Sample_Submission.xlsx  
  inflating: __MACOSX/MessagePolarity_ParticipantsData/._Sample_Submission.xlsx  
  inflating: MessagePolarity_ParticipantsData/Train.csv  
  inflating: __MACOSX/MessagePolarity_ParticipantsData/._Train.csv  


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
train = pd.read_csv('MessagePolarity_ParticipantsData/Train.csv')
test = pd.read_csv('MessagePolarity_ParticipantsData/Test.csv')

In [0]:
train.head(5)

,Freq_Of_Word_1,Freq_Of_Word_2,Freq_Of_Word_3,Freq_Of_Word_4,Freq_Of_Word_5,Freq_Of_Word_6,Freq_Of_Word_7,Freq_Of_Word_8,Freq_Of_Word_9,Freq_Of_Word_10,Freq_Of_Word_11,Freq_Of_Word_12,Freq_Of_Word_13,Freq_Of_Word_14,Freq_Of_Word_15,Freq_Of_Word_16,Freq_Of_Word_17,Freq_Of_Word_18,Freq_Of_Word_19,Freq_Of_Word_20,Freq_Of_Word_21,Freq_Of_Word_22,Freq_Of_Word_23,Freq_Of_Word_24,Freq_Of_Word_25,Freq_Of_Word_26,Freq_Of_Word_27,Freq_Of_Word_28,Freq_Of_Word_29,Freq_Of_Word_30,Freq_Of_Word_31,Freq_Of_Word_32,Freq_Of_Word_33,Freq_Of_Word_34,Freq_Of_Word_35,Freq_Of_Word_36,Freq_Of_Word_37,Freq_Of_Word_38,Freq_Of_Word_39,Freq_Of_Word_40,Freq_Of_Word_41,Freq_Of_Word_42,Freq_Of_Word_43,Freq_Of_Word_44,Freq_Of_Word_45,Freq_Of_Word_46,Freq_Of_Word_47,Freq_Of_Word_48,Freq_Of_Word_49,Freq_Of_Word_50,TotalEmojiCharacters,LengthOFFirstParagraph,StylizedLetters,IsGoodNews
0,-0.351864,2.620660,1.253645,-0.039223,-0.465210,-0.353977,-0.304257,-0.240708,-0.318797,-0.352968,-0.320591,-0.635215,-0.307555,-0.184379,-0.189867,-0.333610,-0.321557,-0.362461,-0.707520,1.065873,0.953319,-0.132948,-0.291683,0.764765,-0.331713,-0.298801,-0.213736,-0.228932,-0.164988,-0.233062,-0.151049,1.312337,-0.183048,-0.152367,-0.236402,-0.244919,-0.329374,-0.055522,-0.181398,-0.187171,-0.133876,-0.168998,-0.208657,-0.120401,-0.323019,-0.205212,-0.079531,-0.118688,0.079303,0.157385,-0.028751,-0.046474,0.222453,1
1,-0.351864,-0.318036,-0.561952,-0.039223,-0.465210,-0.353977,-0.304257,3.837751,-0.318797,-0.352968,-0.320591,-0.635215,-0.307555,-0.184379,-0.189867,-0.333610,-0.321557,-0.362461,-0.973161,-0.191111,-0.707575,-0.132948,-0.291683,-0.197881,-0.331713,-0.298801,-0.213736,-0.228932,-0.164988,-0.233062,-0.151049,-0.150852,-0.183048,-0.152367,-0.236402,-0.244919,3.675591,-0.055522,-0.181398,-0.187171,-0.133876,-0.168998,15.211013,-0.120401,-0.323019,-0.205212,-0.079531,-0.118688,-0.151911,-0.453742,-0.107383,-0.195476,-0.408024,0
2,-0.351864,-0.318036,-0.561952,-0.039223,-0.465210,-0.353977,-0.304257,-0.240708,-0.318797,-0.352968,-0.320591,-0.635215,-0.307555,-0.184379,-0.189867,-0.333610,-0.321557,-0.362461,-0.973161,-0.191111,-0.707575,-0.132948,-0.291683,-0.197881,-0.331713,-0.298801,-0.213736,-0.228932,-0.164988,-0.233062,-0.151049,-0.150852,-0.183048,-0.152367,-0.236402,-0.244919,-0.329374,-0.055522,-0.181398,-0.187171,-0.133876,-0.168998,-0.208657,-0.120401,-0.323019,-0.205212,-0.079531,-0.118688,-0.151911,-0.453742,-0.107383,-0.187634,-0.392578,0
3,1.210190,2.682528,1.291868,-0.039223,0.221744,-0.353977,-0.304257,0.859101,-0.318797,2.374782,-0.320591,2.030243,-0.307555,1.287387,-0.189867,0.289792,-0.321557,-0.362461,0.128969,-0.191111,3.977893,-0.132948,-0.291683,-0.197881,-0.331713,-0.298801,-0.213736,-0.228932,-0.164988,-0.233062,-0.151049,-0.150852,-0.183048,-0.152367,-0.236402,-0.244919,-0.329374,-0.055522,-0.181398,-0.187171,-0.133876,-0.168998,-0.208657,-0.120401,-0.323019,-0.205212,-0.079531,-0.118688,-0.151911,0.340723,1.334201,2.270899,0.602985,1
4,-0.351864,-0.318036,-0.561952,-0.039223,-0.465210,-0.353977,-0.304257,-0.240708,-0.318797,-0.352968,-0.320591,-0.635215,-0.307555,-0.184379,-0.189867,-0.333610,-0.321557,-0.362461,-0.973161,-0.191111,-0.707575,-0.132948,-0.291683,-0.197881,-0.331713,-0.298801,0.101269,-0.228932,0.644464,-0.233062,-0.151049,-0.150852,-0.183048,-0.152367,-0.236402,-0.244919,-0.329374,-0.055522,-0.181398,-0.187171,-0.133876,-0.168998,-0.208657,-0.120401,-0.323019,-0.205212,-0.079531,-0.118688,-0.151911,0.930461,-0.028751,-0.113133,-0.128592,0


In [0]:
test.head(5)

,Freq_Of_Word_1,Freq_Of_Word_2,Freq_Of_Word_3,Freq_Of_Word_4,Freq_Of_Word_5,Freq_Of_Word_6,Freq_Of_Word_7,Freq_Of_Word_8,Freq_Of_Word_9,Freq_Of_Word_10,Freq_Of_Word_11,Freq_Of_Word_12,Freq_Of_Word_13,Freq_Of_Word_14,Freq_Of_Word_15,Freq_Of_Word_16,Freq_Of_Word_17,Freq_Of_Word_18,Freq_Of_Word_19,Freq_Of_Word_20,Freq_Of_Word_21,Freq_Of_Word_22,Freq_Of_Word_23,Freq_Of_Word_24,Freq_Of_Word_25,Freq_Of_Word_26,Freq_Of_Word_27,Freq_Of_Word_28,Freq_Of_Word_29,Freq_Of_Word_30,Freq_Of_Word_31,Freq_Of_Word_32,Freq_Of_Word_33,Freq_Of_Word_34,Freq_Of_Word_35,Freq_Of_Word_36,Freq_Of_Word_37,Freq_Of_Word_38,Freq_Of_Word_39,Freq_Of_Word_40,Freq_Of_Word_41,Freq_Of_Word_42,Freq_Of_Word_43,Freq_Of_Word_44,Freq_Of_Word_45,Freq_Of_Word_46,Freq_Of_Word_47,Freq_Of_Word_48,Freq_Of_Word_49,Freq_Of_Word_50,TotalEmojiCharacters,LengthOFFirstParagraph,StylizedLetters
0,-0.351864,-0.318036,-0.561952,-0.039223,-0.465210,-0.353977,-0.304257,-0.240708,-0.318797,-0.352968,-0.320591,0.801719,-0.307555,-0.184379,-0.189867,1.367757,-0.321557,2.168698,-0.973161,-0.191111,2.736594,-0.132948,-0.291683,2.485239,-0.331713,-0.298801,-0.213736,-0.228932,-0.164988,-0.233062,-0.151049,-0.150852,-0.183048,-0.152367,-0.236402,-0.244919,-0.329374,-0.055522,-0.181398,-0.187171,-0.133876,-0.168998,-0.208657,-0.120401,-0.323019,1.284346,-0.079531,-0.118688,-0.151911,-0.453742,-0.002540,-0.007263,-0.283052
1,-0.351864,-0.318036,-0.561952,-0.039223,0.007071,-0.353977,-0.304257,-0.240708,-0.318797,0.570271,-0.320591,-0.635215,-0.307555,-0.184379,-0.189867,-0.333610,-0.321557,-0.362461,-0.600132,-0.191111,-0.419104,-0.132948,-0.291683,-0.197881,3.951345,2.705426,-0.213736,-0.228932,-0.164988,-0.233062,-0.151049,-0.150852,0.479145,-0.152367,-0.236402,-0.244919,2.640600,-0.055522,-0.181398,-0.187171,8.109520,0.212177,-0.208657,0.291256,-0.323019,-0.205212,-0.079531,-0.118688,1.070723,1.147411,-0.054961,-0.148423,0.049739
2,0.006107,-0.318036,-0.351725,-0.039223,-0.307783,0.027319,-0.304257,-0.240708,3.430188,-0.352968,-0.320591,-0.262270,-0.307555,-0.184379,-0.189867,-0.333610,-0.321557,-0.362461,-0.718823,-0.191111,-0.515261,-0.132948,-0.291683,-0.197881,-0.010906,0.475015,-0.149569,-0.228932,-0.164988,-0.233062,-0.151049,-0.150852,0.499211,-0.152367,-0.236402,-0.244919,0.165622,-0.055522,-0.181398,-0.187171,-0.133876,-0.168998,-0.208657,-0.120401,-0.323019,-0.205212,-0.079531,-0.118688,0.121342,0.068771,-0.054961,-0.062158,0.322149
3,-0.351864,-0.318036,-0.561952,-0.039223,-0.465210,-0.353977,2.089042,-0.240708,-0.318797,-0.352968,-0.320591,0.505557,-0.307555,-0.184379,-0.189867,-0.333610,1.918833,-0.362461,1.095453,-0.191111,1.119408,-0.132948,-0.291683,-0.197881,-0.331713,-0.298801,-0.213736,-0.228932,-0.164988,-0.233062,-0.151049,-0.150852,-0.183048,-0.152367,-0.236402,-0.244919,-0.329374,-0.055522,-0.181398,-0.187171,-0.133876,-0.168998,-0.208657,-0.120401,-0.323019,-0.205212,-0.079531,-0.118688,0.065290,0.304055,-0.054961,0.008422,-0.204418
4,0.624420,0.609973,-0.561952,-0.039223,0.393482,0.685920,-0.304257,1.134053,-0.318797,-0.352968,-0.320591,-0.635215,-0.307555,-0.184379,-0.189867,-0.333610,-0.321557,-0.362461,-0.973161,-0.191111,-0.707575,-0.132948,-0.291683,-0.197881,0.005978,0.042589,-0.213736,-0.228932,0.284707,-0.233062,0.543940,-0.150852,-0.183048,-0.152367,0.437249,0.489245,-0.329374,-0.055522,-0.181398,-0.187171,-0.133876,-0.168998,1.087114,-0.120401,-0.323019,-0.205212,-0.079531,-0.118688,-0.151911,-0.453742,-0.107383,-0.191555,-0.313944


In [0]:
train.duplicated().sum(), test.duplicated().sum()

(4, 3)

In [0]:
train.shape,test.shape

((947, 54), (527, 53))

In [0]:
train=train=train.drop_duplicates(keep='first')
test=test.drop_duplicates(keep='first')

In [0]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

In [0]:
df = train.append(test,ignore_index=True)

In [0]:

train_df = df[df['IsGoodNews'].isnull()!=True]
test_df = df[df['IsGoodNews'].isnull()==True]
test_df.drop(['IsGoodNews'], axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
X = train_df.drop(labels=['IsGoodNews'], axis=1)
y = train_df['IsGoodNews'].values

X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.25, random_state=1, stratify=y)

In [0]:

xgb = XGBClassifier(random_state=42)
xgb.fit(X_train, y_train)
y_pred = xgb.predict_proba(X_cv)

log_loss(y_cv, y_pred)

0.206963767253112

In [0]:

Xtest = test_df

In [0]:

err = []
y_pred_tot = []

fold = StratifiedKFold(n_splits=4, shuffle=False, random_state=42)

for train_index, test_index in fold.split(X, y):
    
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    xgb = XGBClassifier(random_state=42,n_estimators=2000 )
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict_proba(X_test)
    
    print("Log Loss:", log_loss(y_test, y_pred))

    err.append(log_loss(y_test, y_pred))
    p = xgb.predict(Xtest)
    y_pred_tot.append(p)

Log Loss: 0.2485479676244379
Log Loss: 0.2854641211574147
Log Loss: 0.2824163270936168
Log Loss: 0.19128175487753507


In [0]:
np.mean(err, 0)

0.2519275426882511

In [0]:
np.mean(y_pred_tot, 0)

array([0.5 , 0.  , 0.  , 1.  , 0.  , 0.75, 0.25, 1.  , 1.  , 0.  , 0.75,
       1.  , 0.  , 1.  , 0.  , 1.  , 0.5 , 1.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 1.  , 1.  , 0.5 , 0.  , 1.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 1.  , 0.25, 1.  , 1.  , 0.  , 0.  , 0.  , 0.  , 1.  ,
       1.  , 1.  , 1.  , 0.75, 1.  , 0.25, 0.  , 0.  , 1.  , 0.  , 0.  ,
       0.75, 0.25, 0.  , 1.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 1.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.25, 1.  , 1.  ,
       0.  , 1.  , 1.  , 0.  , 0.  , 0.  , 0.  , 1.  , 1.  , 0.  , 1.  ,
       0.75, 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 1.  , 0.25, 0.  , 1.  ,
       0.  , 0.  , 1.  , 0.  , 0.25, 0.  , 0.  , 1.  , 1.  , 1.  , 1.  ,
       0.  , 1.  , 0.  , 0.  , 1.  , 0.  , 0.  , 1.  , 1.  , 0.  , 1.  ,
       0.  , 0.  , 1.  , 0.5 , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 1.  ,
       1.  , 0.  , 0.25, 1.  , 0.  , 1.  , 1.  , 1.  , 0.  , 0.  , 0.  ,
       0.25, 1.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.

In [0]:
y_pred = np.mean(y_pred_tot, 0)
y_pred.round().astype(int)

In [0]:
sub = pd.DataFrame(y_pred.round().astype(int))
sub.head(3)

,0
0,0
1,0
2,0
3,1
4,0


In [0]:
sub['IsGoodNews']=sub[0]
del sub[0]
sub['IsGoodNews'].value_counts()

0    337
1    190
Name: IsGoodNews, dtype: int64

In [0]:
sub.to_excel('xgtrick8.xlsx', index=False)

In [0]:
from google.colab import files
files.download('xgtrick8.xlsx')

In [0]:
#.95082

In [0]:
X = train_df.drop(labels=['IsGoodNews'], axis=1)
y = train_df['IsGoodNews'].values

X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.25, random_state=1, stratify=y)

In [0]:
xgb = XGBClassifier(random_state=42)
xgb.fit(X_train, y_train)
y_pred = xgb.predict_proba(X_cv)

log_loss(y_cv, y_pred)

0.206963767253112

In [0]:
Xtest = test_df

In [0]:
!pip install catboost
from catboost import CatBoostClassifier
err = []
y_pred_tot = []

fold = StratifiedKFold(n_splits=4, shuffle=False, random_state=None)

for train_index, test_index in fold.split(X, y):
    
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    xgb = CatBoostClassifier(random_state=42,n_estimators=2000 )
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_test)
    
    print("Log Loss:", log_loss(y_test, y_pred))

    err.append(log_loss(y_test, y_pred))
    p = xgb.predict(Xtest)
    y_pred_tot.append(p)

Streaming output truncated to the last 5000 lines.
1006:	learn: 0.1233169	total: 5.86s	remaining: 5.78s
1007:	learn: 0.1232505	total: 5.86s	remaining: 5.77s
1008:	learn: 0.1232028	total: 5.87s	remaining: 5.76s
1009:	learn: 0.1231163	total: 5.88s	remaining: 5.76s
1010:	learn: 0.1230914	total: 5.88s	remaining: 5.75s
1011:	learn: 0.1229779	total: 5.89s	remaining: 5.75s
1012:	learn: 0.1228787	total: 5.89s	remaining: 5.74s
1013:	learn: 0.1227911	total: 5.9s	remaining: 5.74s
1014:	learn: 0.1227545	total: 5.9s	remaining: 5.73s
1015:	learn: 0.1227168	total: 5.91s	remaining: 5.72s
1016:	learn: 0.1226045	total: 5.91s	remaining: 5.72s
1017:	learn: 0.1225307	total: 5.92s	remaining: 5.71s
1018:	learn: 0.1225032	total: 5.93s	remaining: 5.71s
1019:	learn: 0.1224703	total: 5.93s	remaining: 5.7s
1020:	learn: 0.1223484	total: 5.94s	remaining: 5.7s
1021:	learn: 0.1223305	total: 5.95s	remaining: 5.69s
1022:	learn: 0.1222842	total: 5.96s	remaining: 5.69s
1023:	learn: 0.1222542	total: 5.97s	remaining: 5.69s

In [0]:
np.mean(err, 0)

2.588642761467744

In [0]:
np.mean(y_pred_tot, 0)

array([1.  , 0.  , 0.  , 1.  , 0.  , 0.75, 0.  , 1.  , 1.  , 0.  , 0.75,
       1.  , 0.  , 1.  , 0.  , 1.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 1.  , 1.  , 0.75, 0.  , 1.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 1.  , 0.  , 1.  , 1.  , 0.  , 0.  , 0.  , 0.  , 1.  ,
       1.  , 1.  , 1.  , 1.  , 1.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ,
       1.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 1.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 1.  , 1.  ,
       0.  , 1.  , 1.  , 0.  , 0.  , 0.  , 0.  , 1.  , 1.  , 0.  , 1.  ,
       1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.75, 1.  , 0.  , 0.  , 1.  ,
       0.  , 0.  , 1.  , 0.  , 0.25, 0.  , 0.  , 1.  , 1.  , 1.  , 1.  ,
       0.  , 1.  , 0.  , 0.  , 1.  , 0.  , 0.  , 1.  , 1.  , 0.  , 1.  ,
       0.  , 0.  , 1.  , 1.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 1.  ,
       1.  , 0.  , 0.  , 1.  , 0.  , 1.  , 1.  , 1.  , 0.  , 0.  , 0.  ,
       0.  , 1.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.

In [0]:
y_pred = np.mean(y_pred_tot, 0)
y_pred.round().astype(int)

array([1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0,

In [0]:
sub = pd.DataFrame(y_pred.round().astype(int))
sub.head(3)

,0
0,1
1,0
2,0


In [0]:
sub['IsGoodNews']=sub[0]
del sub[0]
sub['IsGoodNews'].value_counts()

0    332
1    195
Name: IsGoodNews, dtype: int64

In [0]:
sub.to_excel('xgtrick18.xlsx', index=False)

In [0]:
from google.colab import files
files.download('xgtrick18.xlsx')